Hook函数机制：不改变主体，实现额外功能，像一个挂件，挂钩，hook  

1.torch.Tensor.register_hook  
功能：注册一个反向传播hook函数  
Hook函数仅一个输入参数，为张量的梯度

In [24]:
import torch
import torch.nn as nn

In [16]:
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)
a = torch.add(w, x)
b = torch.add(w, 1)
y = torch.mul(a, b)
a_grad = list()
def grad_hook(grad):
    a_grad.append(grad)
handle = a.register_hook(grad_hook)
y.backward()
print(w.grad, x.grad, a.grad, b.grad, y.grad)
print(a_grad[0])
print(a_grad)
handle.remove()

tensor([5.]) tensor([2.]) None None None
tensor([2.])
[tensor([2.])]


In [39]:
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)
a = torch.add(w, x)
b = torch.add(w, 1)
y = torch.mul(a, b)
a_grad = list()
def grad_hook(grad):
    grad *=2
    # return grad * 3
handle = w.register_hook(grad_hook)
y.backward()
print(w.grad)
handle.remove()

tensor([10.])


Module.register_forward_hook  
功能：注册module的前向传播hook函数  
参数：  
* module:当前网络层  
* input:当前网络层输入数据  
* output:当前网络层输出数据

Module.register_forward_pre_hook  
功能：注册module前向传播前的hook函数  
参数：  
* module:当前网络层  
* input:当前网络层输入数据

Module.register_backward_hook  
功能:注册module反向传播的hook函数  
参数:  
* module:当前网络层  
* grad_input:当前网络层输入梯度数据  
* grad_output:当前网络层输出梯度数据  


In [34]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 3)
        self.pool1 = nn.MaxPool2d(2, 2)
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        return x

In [35]:
def forward_hook(module, data_input, data_output):
    fmap_block.append(data_output)
    input_block.append(data_input)
    
    
def forward_pre_hook(module, data_input):
    print('forward_pre_hook input:{}'.format(data_input))
    
    
def backward_hook(module, grad_input, grad_output):
    print('backward hook input:{}'.format(grad_input))
    print('backward hook output:{}'.format(grad_output))

In [36]:
#初始化网络
net = Net()
net.conv1.weight[0].detach().fill_(1)
net.conv1.weight[1].detach().fill_(2)
net.conv1.bias.data.detach().zero_()
#注册hook
fmap_block = list()
input_block = list()
net.conv1.register_forward_hook(forward_hook)
net.conv1.register_forward_pre_hook(forward_pre_hook)
net.conv1.register_backward_hook(backward_hook)

fake_img = torch.ones((1, 1, 4, 4))
output = net(fake_img)
loss_fnc = nn.L1Loss()
target = torch.randn_like(output)
loss = loss_fnc(target, output)
loss.backward()



forward_pre_hook input:(tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]]),)
backward hook input:(None, tensor([[[[0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000]]],


        [[[0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000],
          [0.5000, 0.5000, 0.5000]]]]), tensor([0.5000, 0.5000]))
backward hook output:(tensor([[[[0.5000, 0.0000],
          [0.0000, 0.0000]],

         [[0.5000, 0.0000],
          [0.0000, 0.0000]]]]),)


In [37]:
print(output.shape, output)
print(fmap_block[0].shape, fmap_block[0])
print(input_block[0][0].shape, input_block[0])

torch.Size([1, 2, 1, 1]) tensor([[[[ 9.]],

         [[18.]]]], grad_fn=<MaxPool2DWithIndicesBackward>)
torch.Size([1, 2, 2, 2]) tensor([[[[ 9.,  9.],
          [ 9.,  9.]],

         [[18., 18.],
          [18., 18.]]]], grad_fn=<MkldnnConvolutionBackward>)
torch.Size([1, 1, 4, 4]) (tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]]),)


CAM:类激活图，class activation map  
判断类别究竟是根据图片哪个判断确定的。需要在网络后加一个GAP

Grad-CAM:CAM改进版，利用梯度作为特征图权重